<a href="https://colab.research.google.com/github/Elimeleth/aprende_a_crear_asistentes/blob/main/Mirasc0p3_RAG_Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qqqU mirascope qdrant-client sentence-transformers "mirascope[groq]" groq pydantic

In [ ]:
%pip show mirascope

Name: mirascope
Version: 1.1.0
Summary: LLM abstractions that aren't obstructions
Home-page: https://mirascope.io
Author: 
Author-email: William Bakst <william@mirascope.io>, Brendan Kao <brendan@mirascope.io>
License: MIT License
        
        Copyright (c) 2023 Mirascope, Inc.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY 

In [ ]:
from qdrant_client import QdrantClient, models
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

client = QdrantClient(api_key=userdata.get("QDRANT_API_KEY"), url=userdata.get("QDRANT_URL"))
collection_name="gogh"

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from mirascope.core import groq, prompt_template, BaseTool
from pydantic import ConfigDict, Field, BaseModel
from typing import List

In [ ]:
class QdrantRetrieval(BaseModel):
    """
    Busca informacion relacionada a la query del cliente.

    Args:
      query: La query que el cliente ha realizado.

    Returns:
      Un string con la informacion relacionada a la query.
    """
    query: str= Field(..., examples=["Precio del brunch", "Daypass pizza", "horarios"])

    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {"query": "precio del brunch"},
                {"query": "daypass pizza"},
                {"query": "horarios"},
            ]
        }
    )

    def call(self) -> str:
      return qdrant_retrieval(self.query)

def qdrant_retrieval(query: str) -> str:

    try:
      points = client.query_points(
          collection_name,
          prefetch=[
              models.Prefetch(
                  query=model.encode(query).tolist(),
                  using="dense-vector",
                  limit=50,
                  score_threshold=.35,
              )
          ],

          query=model.encode(query, output_value="token_embeddings").tolist(),
          using="output-token-embeddings",
          limit=5,
          score_threshold=.7,
      ).points

      return "\n".join([point.payload["text"] for point in points])
    except Exception as e:
      return str(e)

In [ ]:
from groq import Groq
from groq.types.chat import ChatCompletionMessageParam

llm = Groq(
    api_key=os.environ["GROQ_API_KEY"],
)

In [ ]:
class Assistant(BaseModel):
    history: list[ChatCompletionMessageParam] = []

    @groq.call(model="llama3-groq-70b-8192-tool-use-preview", stream=True, client=llm, tools=[QdrantRetrieval])
    @prompt_template(
        """
        SYSTEM:
        Eres Marina, encargado del restaurante Rancho Santa Marina.
        Tu tarea es responder preguntas sobre el restaurante.
        Usa la tool `qdrant_retrieval` para obtener la informacion necesaria sobre la pregunta del cliente.
        Finalmente provee una respuesta breve no mas de 3 lineas.

        MESSAGES:
        {self.history}

        USER:
        {question}
        """
    )
    def _step(self, question: str): ...

    def run(self, user_query: str) -> str:
      stream = self._step(user_query)
      result, tools_and_outputs = "", []
      for chunk, tool in stream:
          if tool:
              tools_and_outputs.append((tool, tool.call()))
          else:
              result += chunk.content
      if stream.user_message_param:
          self.history.append(stream.user_message_param)
      self.history.append(stream.message_param)
      if tools_and_outputs:
          self.history += stream.tool_message_params(tools_and_outputs)
          return self.run("")
      return result

In [ ]:
Assistant().run("los menoras pagan daypass verano?")

'Sí, los menores pagan el Daypass de Verano. La promoción activa incluye un taller de pizza familiar con un precio especial. El menú del taller incluye: 1 pizza, ensalada, postre y agua del día por persona. El precio es de $400 pesos por persona, menor y acompañante. La experiencia dura 4 horas.'